## Install Dependancies

In [77]:
!pip install -q transformers torch datasets rouge-score sacrebleu grapheme gradio

## Import libraries

In [78]:
import os
import torch
import string
import numpy as np
import pandas as pd
import collections
from pathlib import Path
from datetime import datetime
from rouge_score import rouge_scorer
import grapheme
import seaborn as sns
from sacrebleu.metrics import BLEU
from sklearn.metrics import f1_score
from transformers import Trainer, TrainingArguments, default_data_collator, EarlyStoppingCallback, AutoTokenizer, AutoModelForQuestionAnswering
import tensorflow as tf
from datasets import Dataset
from tqdm.auto import tqdm
from rouge_score import rouge_scorer
import nltk
import matplotlib.pyplot as plt
import json
import gradio as gr
import os
from transformers import TrainerCallback
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


## Load datasets

In [4]:
train = pd.read_csv('/content/drive/MyDrive/Research_folder/data/train.csv')
val = pd.read_csv('/content/drive/MyDrive/Research_folder/data/dev.csv')
test = pd.read_csv('/content/drive/MyDrive/Research_folder/data/test.csv')

In [5]:
# For controlling the data ratio
# percentage = 0.75
# size = 100
# train = train_full.sample(frac=percentage, random_state=42)  # random_state for reproducibility
# print(f"Loaded {len(train)} rows ({percentage*100}% of {len(train_full)} total)")

In [6]:
train.head(4)

,Index,id,context,question,answer_text,answer_start
0,23174,56d9b69cdc89441400fdb728,කොයෝට් සහ විශාල බළලුන් ද බල්ලන්ට පහර දෙන බව දන...,බල්ලන් අනුභව කරන උරග වර්ග මොනවාද?,කිඹුලන් සහ පිඹුරන්,415
1,15815,572fd2b4b2c2fd14005684b5,පළමු නාවික සටන් කිහිපය රෝමයට ව්යසනකාරී විය. කෙ...,රෝමවරුන්ට එරෙහිව කිහිප වතාවක් පරාජයට පත්වීමෙන්...,සැන්තිපස්,344
2,32102,570c6902fed7b91900d459bb,වෝල්ටර්ගේ මරණයෙන් පසු නොර්විච් හි බිෂොප් සහ ඔහ...,ජෝන් ඩි ග්රේට ඔවුන්ගේ සහයෝගය වෙනස් කිරීමට ජෝන්...,කැන්ටබරි පරිච්ඡේදය,566
3,22806,56d29d4659d6e41400146144,මහායාන සූත්ර යනු මහායාන බෞද්ධ සම්ප්රදාය විසින්...,බුදුන් වහන්සේගේ මුල් ඉගැන්වීම් ලෙස සැලකෙන්නේ ම...,මහායාන සූත්ර,0


In [7]:
# translated test set
test.head(4)

,Index,id,context,question,answer_text,answer_start,question_type,question_length,context_length,answer_length
0,17322,57309733396df919000961dc,බොහෝ ගුවන්තොටුපළවල් එම ස්ථානයේදීම රූගත කිරීම් ...,"ගොඩබිම, රූගත කිරීම් කළ හැක්කේ කොහේද?",සියලුම පොදු ප්රදේශ,149,Where,6,104,3
1,4765,5727784ff1498d1400e8f902,ඩිමොක්රටික් පක්ෂය ව්යවස්ථාදායකයට සහ ආණ්ඩුකාර ධ...,ද්විවාර්ගික ජනතාවාදී එකමුතුවක් ආණ්ඩුකාර ධුරය ල...,1896,142,Which,9,75,1
2,2693,5726aefadd62a815002e8cec,"1500 ගණන්වලදී, ගැලන් ප්රසිද්ධියේ විවේචනය කළ පළ...",1500 ගණන්වල ගැලන් විවේචනය කළේ කවුද?,පැරසෙල්සස්,58,Who,6,141,1
3,26567,56e963a699e8941900975f8b,"2009-10 පාසල් වර්ෂය තුළ, බර්න්හි පන්තිවලට සහභා...",ප්රාථමික පන්ති කීයක් තිබුණාද?,266,316,How much,4,105,1


In [8]:
# translated validation set
val.head(4)

,Index,id,context,question,answer_text,answer_start
0,31226,570b38466b8089140040f814,"1689 සිට, උතුරු ඇමරිකාව පාලනය කිරීම සඳහා මහා බ...",ප්රංශ සහ ඉන්දියානු යුද්ධයේදී සටන් කළ අනාගත ඇමර...,ජෝර්ජ් වොෂින්ටන්,371
1,37977,56e17e6ee3433e1400422f7f,ගැටලුවක් සහ අවස්ථාවක් අතර වෙනස තවදුරටත් උද්දීප...,සංචාරක විකුණුම්කරුගේ ගැටලුව ජර්මනියේ විශාලතම න...,2000,167
2,1375,57277421708984140094ddf7,1456 දී ඔටෝමන්වරුන්ට එරෙහිව බෙල්ග්රේඩ් වටලෑමේද...,බෙල්ග්රේඩ් වැටලීමේදී මුස්ලිම්වරුන්ට එරෙහිව සටන...,කැපිස්ට්රානෝහි ශාන්ත ජෝන්,203
3,11434,572ac29334ae481900deacc4,බොහෝ විශ්ලේෂණයන් කෙරීගේ ඡන්ද වාර්තාව සෙනෙට් ප්...,කෙරී &#39;ඉහළම සෙනෙට් ලිබරල්&#39; ලෙස හැඳින්වූ...,2003,201


In [9]:
print(f"Translated Train set shape:{train.shape}")
print(f"Translated Validation set shape:{val.shape}")
print(f"Translated Test set shape:{test.shape}")

Translated Train set shape:(13500, 6)
Translated Validation set shape:(1250, 6)
Translated Test set shape:(1250, 10)


## Model loading

In [10]:
model_name = "deepset/xlm-roberta-large-squad2"
# Load the model and tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

## Feature Extraction

In [12]:
# Set up configuration for the tokenization
batch_size = 16
max_length = 512
doc_stride = 128  # Stride value to handle document overflow
pad_on_right = tokenizer.padding_side == "right"  # Check if padding is applied on the right side

In [13]:
def prepare_train_features(examples):
    # Remove unnecessary left whitespace from questions to avoid truncation issues
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize the examples, applying truncation, padding, and managing overflows with stride
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Map features back to their corresponding examples
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Initialize lists to store start and end positions for answers
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # Find the CLS token index for unanswered examples
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        # If no answer, label with CLS token index
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Calculate start and end token indices for the answer span
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Find the start and end token indices within the current span
            token_start_index = next(i for i, s in enumerate(sequence_ids) if s == (1 if pad_on_right else 0))
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Label with CLS index if answer is out of span
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Adjust start and end positions based on answer span
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [14]:
def convert_answers(r):
    # Extract the starting position
    start = r[0]
    # Extract the answer text
    text = r[1]
    # Return the answer in a dictionary
    return {
        'answer_start': [start],  # Start index of the answer as a list.
        'text': [text]           # The text of the answer as a list.
    }

In [15]:
train = train.sample(frac=1, random_state=42)
valid = val.sample(frac=1, random_state=42)

# Combines 'answer_start' and 'answer_text' into a structured dictionary.
train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)
valid['answers'] = valid[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

# Convert DataFrame into Hugging Face's Dataset
train_dataset = Dataset.from_pandas(train)
valid_dataset = Dataset.from_pandas(valid)

<ipython-input-14-521c13062572>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start = r[0]
<ipython-input-14-521c13062572>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = r[1]
<ipython-input-14-521c13062572>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start = r[0]
<ipython-input-14-521c13062572>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alway

In [16]:
train_dataset[0]

{'Index': 17926,
 'id': '5730e690f6cb411900e2452c',
 'context': 'හවුල්කාරිත්වයන් ගොඩනැගීම විස්තර කරනු ලබන්නේ අන්යෝන්ය ප්රතිලාභ සහ ආරක්ෂාව සඳහා සබඳතා වර්ධනය කිරීම, මඟ පෙන්වීම සහ පවත්වා ගැනීම සඳහා ජාත්යන්තර ගුවන් භටයින් සහ අනෙකුත් අදාළ ක්රියාකාරීන් සමඟ අන්තර් ක්රියා කරන ගුවන් භටයින් ලෙසය. හවුල්කාරිත්වයන් ගොඩනැගීම යනු අන්යෝන්ය ප්රතිලාභ සහ ආරක්ෂාව සඳහා සබඳතා වර්ධනය කිරීම, මඟ පෙන්වීම සහ පවත්වා ගැනීමයි. එබැවින් එය සහජයෙන්ම අන්තර් පුද්ගල සහ හරස් සංස්කෘතික කටයුත්තකි. වචන සහ ක්රියාවන් යන දෙකෙන්ම, අන්තර්ක්රියාවෙන් බහුතරයක් අන්යෝන්ය ප්රතිලාභ සඳහා විශ්වාසය පදනම් කරගත් සබඳතා ගොඩනැගීමට කැපවී ඇත. එයට විදේශීය හවුල්කරුවන් මෙන්ම දේශීය හවුල්කරුවන් ද ඇතුළත් වන අතර විදේශීය රජයන්, හමුදා සහ ජනගහනය මෙන්ම එක්සත් ජනපද රජයේ දෙපාර්තමේන්තු, ආයතන, කර්මාන්ත සහ රාජ්ය නොවන සංවිධාන සමඟ සහයෝගීතාවය අවධාරණය කරයි. හවුල්කාරිත්ව උත්සාහයන් වඩා හොඳින් පහසු කිරීම සඳහා, ගුවන් භටයින් අදාළ භාෂාව, කලාපය සහ සංස්කෘතිය පිළිබඳ දක්ෂ විය යුතුය.',
 'question': 'මෙම ගොඩනැගිලි හවුල්කාරිත්ව මෙහෙයුම් සඳහා ගුවන් භටයින් නිපුණ විය යුතු යැයි අපේක

In [17]:
# Tokenize the dataset
tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/13500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

## Training Process

In [18]:
class ComputeMetricsCallback(TrainerCallback):
    def __init__(self):
        self.train_accuracies = []
        self.val_accuracies = []
        self.epoch_train_losses = []  # Stores last loss of each epoch
        self.current_epoch_train_losses = []  # Temporary storage
        self.val_losses = []
        self.completed_epochs = 0

    def on_epoch_begin(self, args, state, control, **kwargs):
        """Reset training loss tracking at start of each epoch"""
        self.current_epoch_train_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return

        # Track all training losses during epoch
        if "loss" in logs:
            self.current_epoch_train_losses.append(logs["loss"])

        # Capture validation metrics at epoch end
        if "eval_loss" in logs:
            self.val_losses.append(logs["eval_loss"])
        if "eval_accuracy" in logs:
            self.val_accuracies.append(logs["eval_accuracy"])

    def on_epoch_end(self, args, state, control, **kwargs):
        """Store the final training loss/accuracy of the completed epoch"""
        if self.current_epoch_train_losses:
            # Store last training loss of the epoch
            self.epoch_train_losses.append(self.current_epoch_train_losses[-1])

            # Optional: Calculate epoch-level training accuracy if available
            if hasattr(state, 'train_metrics') and 'accuracy' in state.train_metrics:
                self.train_accuracies.append(state.train_metrics['accuracy'])

        self.completed_epochs += 1

In [19]:
# Initialize callback
metrics_callback = ComputeMetricsCallback()

# Define training arguments
%env WANDB_DISABLED=True
args = TrainingArguments(
    f"sinhala-qa",
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.1,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


env: WANDB_DISABLED=True


In [20]:
# Initialize the data collator
data_collator = default_data_collator

# Load model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Set up early stopping
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

# Trainer setup
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=default_data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping, metrics_callback],  # Add accuracy tracking
)

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-20-caf4fd5febc9>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()  # Start training the model

Epoch,Training Loss,Validation Loss
1,2.107900,2.021051
2,1.905700,1.941053
3,1.792100,1.952261
4,1.559300,1.911640


TrainOutput(global_step=1060, training_loss=1.8498094180844864, metrics={'train_runtime': 4546.0957, 'train_samples_per_second': 14.963, 'train_steps_per_second': 0.233, 'total_flos': 6.29031819908137e+16, 'train_loss': 1.8498094180844864, 'epoch': 4.978248089359201})

In [22]:
# Function to save metrics
def save_metrics(metrics_data, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w") as f:
        json.dump(metrics_data, f)

In [23]:
# Save metrics
metrics_data = {
    "train_losses": metrics_callback.epoch_train_losses,
    "val_losses": metrics_callback.val_losses,
    "train_accuracies": metrics_callback.train_accuracies,
    "val_accuracies": metrics_callback.val_accuracies,
}

In [24]:
# Check exact values at key epochs
print(f"Final Training Loss: {metrics_callback.epoch_train_losses[-1]:.4f}")
print(f"Final Validation Loss: {metrics_callback.val_losses[-1]:.4f}")
print(f"Gap: {metrics_callback.val_losses[-1] - metrics_callback.epoch_train_losses[-1]:.4f}")

Final Training Loss: 1.5593
Final Validation Loss: 1.9116
Gap: 0.3523


In [25]:
# save_metrics(metrics_data, "/content/drive/MyDrive/Research_folder/outputs/metrics/size/XLM-R-75-metrics.json")

In [26]:
#trainer.save_model("Sin-xlm-r-qa")  # Save the trained model

## Validation process

In [27]:
def prepare_validation_features(examples):
    # Remove leading whitespace from questions to avoid unnecessary token space usage
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize questions and contexts while handling long texts with a sliding window (stride)
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],  # Choose question or context based on padding side
        examples["context" if pad_on_right else "question"],  # Opposite choice for the second input
        truncation="only_second" if pad_on_right else "only_first",  # Truncate only the context (or question if needed)
        max_length=max_length,  # Set max sequence length
        stride=doc_stride,  # Define overlap between chunks for long contexts
        return_overflowing_tokens=True,  # Return multiple input chunks for long contexts
        return_offsets_mapping=True,  # Keep track of token positions in original text
        padding="max_length",  # Ensure all sequences have the same length
    )

    # Map each generated tokenized example to its original sample index
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # Create a list to store the example ID corresponding to each tokenized input
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):  # Iterate through tokenized samples
        # Identify whether the sequence contains context (1) or question (0)
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0  # Set correct index based on padding side

        # Find the original example index that this tokenized input corresponds to
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])  # Store example ID

        # Adjust offset mapping: Set offsets to None for non-context tokens
        tokenized_examples["offset_mapping"][i] = [
            (offset if sequence_ids[k] == context_index else None)  # Keep offsets only for context tokens
            for k, offset in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [28]:
validation_features = valid_dataset.map(
    prepare_validation_features,  # Preprocess validation data
    batched=True,  # Process the dataset in batches
    remove_columns=valid_dataset.column_names  # Removes original columns
)

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [29]:
# Print the number of processed validation features
print(f"Number of validation features: {len(validation_features)}")

Number of validation features: 1259


In [30]:
valid_dataset

Dataset({
    features: ['Index', 'id', 'context', 'question', 'answer_text', 'answer_start', 'answers', '__index_level_0__'],
    num_rows: 1250
})

In [31]:
# Create a smaller version of the validation features dataset
valid_feats_small = validation_features.map(
    lambda example: example,  # Lambda function returns the example
    remove_columns=['example_id', 'offset_mapping']  # Remove unnecessary columns for evaluation
)

Map:   0%|          | 0/1259 [00:00<?, ? examples/s]

In [32]:
valid_feats_small

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1259
})

In [33]:
# Make predictions on the validation dataset
raw_predictions = trainer.predict(valid_feats_small)

In [34]:
# Get the raw predictions
raw_predictions[0]

(array([[-0.7404497 , -8.196019  , -8.223466  , ..., -9.34765   ,
         -9.34765   , -9.34765   ],
        [-2.2339582 , -8.117479  , -8.417618  , ..., -9.231927  ,
         -9.231927  , -9.231927  ],
        [-1.3503318 , -8.254151  , -8.56361   , ..., -9.449091  ,
         -9.449091  , -9.449091  ],
        ...,
        [ 0.37089145, -6.8477535 , -3.6149604 , ..., -9.170397  ,
         -9.170397  , -9.170397  ],
        [-1.0252664 , -7.7274213 , -8.141339  , ..., -9.236161  ,
         -9.236161  , -9.236161  ],
        [ 1.2130126 , -8.233382  , -8.345722  , ..., -9.240337  ,
         -9.240337  , -9.240337  ]], dtype=float32),
 array([[ -3.4791274, -11.306823 , -11.1244955, ..., -12.46734  ,
         -12.46734  , -12.46734  ],
        [ -4.458971 , -10.506583 , -10.233945 , ..., -12.022856 ,
         -12.022856 , -12.022856 ],
        [ -3.9809885, -10.629296 , -11.012989 , ..., -12.514319 ,
         -12.514319 , -12.514319 ],
        ...,
        [ -2.245312 ,  -9.452142 ,  -7.

In [35]:
examples = valid_dataset
features = validation_features

# Map example_id to its index in the examples dataset
example_id_to_index = {k: i for i, k in enumerate(examples["id"])}

# Initialize defaultdict to store feature indices per example
features_per_example = collections.defaultdict(list)

# Populate the dictionary with feature indices corresponding to each example_id
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)


In [36]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions

    # Map example to its corresponding feature indices
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} examples with {len(features)} features.")

    # Iterate over all examples
    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]
        min_null_score = None
        valid_answers = []

        context = example["context"]
        # Process each feature associated with the current example
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            # Track minimum null score (if needed)
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            min_null_score = max(min_null_score or float('-inf'), feature_null_score)

            # Get top n start and end logits
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

            # Evaluate each possible answer span
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip invalid spans
                    if start_index >= len(offset_mapping) or end_index >= len(offset_mapping) or offset_mapping[start_index] is None or offset_mapping[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    # Extract the answer text
                    start_char, end_char = offset_mapping[start_index][0], offset_mapping[end_index][1]
                    valid_answers.append({"score": start_logits[start_index] + end_logits[end_index], "text": context[start_char:end_char]})

        # Select the best answer
        best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0] if valid_answers else {"text": "", "score": 0.0}
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [37]:
# Post-process raw predictions to extract the final answers for each example in the validation dataset
final_predictions = postprocess_qa_predictions(valid_dataset, validation_features, raw_predictions.predictions)

Post-processing 1250 examples with 1259 features.


  0%|          | 0/1250 [00:00<?, ?it/s]

In [38]:
# Create a DataFrame with questions and their corresponding predicted answers
prediction = pd.DataFrame([{"questions": x1['question'], "pred_answer": x2} for x1, x2 in zip(valid_dataset, [i for i in final_predictions.values()])])

In [39]:
# Assuming valid_dataset contains the original context, question, and true answer
prediction = pd.DataFrame([
    {
        "questions": x1['question'],
        "pred_answer": x2,
        "answer_text": x1['answer_text']  # Add the true answer here
    }
    for x1, x2 in zip(valid_dataset, [i for i in final_predictions.values()])
])

In [40]:
# Print dataframe with questions predicted answer
prediction

,questions,pred_answer,answer_text
0,අධිරාජ්යයේ ව්යාප්තිය මන්දගාමී කළේ කුමන ආකාරයේ ...,ආගමික හා බුද්ධිමය ගතානුගතිකත්වය,ආගමික හා බුද්ධිමය
1,ප්රභාසංස්ලේෂණයේදී ශාක නිකුත් කරන්නේ කුමක්ද?,ඔක්සිජන්,ඔක්සිජන්
2,අත්අඩංගුවට ගත් බොහෝ විරුද්ධවාදීන් අයත් වූයේ කු...,සහෝදරත්වය,සහෝදරත්වය
3,වෝර්සෝගේ වීර ඉතිහාසය සිහිපත් කරන්නේ කුමක් ද?,බොහෝ ස්ථාන,බොහෝ ස්ථාන
4,ප්රතිජීවක සඳහා අනුමැතිය අවශ්ය වන්නේ කුමක් සඳහාද?,සායනික භාවිතය,සායනික භාවිතය
...,...,...,...
1245,ගෘහ නිර්මාණ ශිල්පය පුරවා ඇති සමාජයේ අවශ්යතාවල ...,ආගමික හා සමාජීය,ආගමික හා සමාජීය අවශ්යතා
1246,සියලුම ජීවීන්ගේ ස්නායු පද්ධතියේ කේන්ද්රය කුමක්ද?,මොළය,මොළය
1247,"තෘෂ්ණාව, අකමැත්ත, අවිද්යාව යන විෂ ත්රිවිධයෙන් ...",බුදුන් වහන්සේ,බුදු කෙනෙක්
1248,මුස්ලිම්වරුන් කොපමණ භාෂා කතා කරනවාද?,සිය ගණනක්,සිය ගණනක්


## Testing process

In [41]:
# Convert the test DataFrame to a Hugging Face Dataset and process the features for validation
test_dataset = Dataset.from_pandas(test)

test_features = test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [42]:
# Remove unnecessary columns ('example_id' and 'offset_mapping') from the test features
test_feats_small = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

Map:   0%|          | 0/1258 [00:00<?, ? examples/s]

In [43]:
test_feats_small

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1258
})

In [44]:
# Make predictions on the test data
test_predictions = trainer.predict(test_feats_small)

In [45]:
# Set the format of the test features dataset, retaining the original columns and format
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

In [46]:
# Post-process the raw predictions on the test dataset to get the final answers
final_test_predictions = postprocess_qa_predictions(test_dataset, test_features, test_predictions.predictions)

Post-processing 1250 examples with 1258 features.


  0%|          | 0/1250 [00:00<?, ?it/s]

In [47]:
# Add the predicted answers to the test DataFrame
test['PredictionString'] = test['id'].apply(lambda r: final_test_predictions[r])
test.head()

,Index,id,context,question,answer_text,answer_start,question_type,question_length,context_length,answer_length,PredictionString
0,17322,57309733396df919000961dc,බොහෝ ගුවන්තොටුපළවල් එම ස්ථානයේදීම රූගත කිරීම් ...,"ගොඩබිම, රූගත කිරීම් කළ හැක්කේ කොහේද?",සියලුම පොදු ප්රදේශ,149,Where,6,104,3,සියලුම පොදු ප්රදේශවල
1,4765,5727784ff1498d1400e8f902,ඩිමොක්රටික් පක්ෂය ව්යවස්ථාදායකයට සහ ආණ්ඩුකාර ධ...,ද්විවාර්ගික ජනතාවාදී එකමුතුවක් ආණ්ඩුකාර ධුරය ල...,1896,142,Which,9,75,1,1896
2,2693,5726aefadd62a815002e8cec,"1500 ගණන්වලදී, ගැලන් ප්රසිද්ධියේ විවේචනය කළ පළ...",1500 ගණන්වල ගැලන් විවේචනය කළේ කවුද?,පැරසෙල්සස්,58,Who,6,141,1,පැරසෙල්සස්
3,26567,56e963a699e8941900975f8b,"2009-10 පාසල් වර්ෂය තුළ, බර්න්හි පන්තිවලට සහභා...",ප්රාථමික පන්ති කීයක් තිබුණාද?,266,316,How much,4,105,1,266
4,7213,5727e51fff5b5019007d97be,"විධායක ශාඛාව ආණ්ඩුකාරවරයා, ඔවුන්ගේ කාර්ය මණ්ඩල...",විධායක අංශයේ කවුද ඉන්නේ?,"ආණ්ඩුකාරවරයා, ඔවුන්ගේ කාර්ය මණ්ඩලය සහ අනෙකුත් ...",13,Who,4,78,10,"ආණ්ඩුකාරවරයා, ඔවුන්ගේ කාර්ය මණ්ඩලය සහ අනෙකුත්..."


## Evaluations

In [48]:
# Function to remove punctuation from a string
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [49]:
# Function to compute Exact Match
def compute_exact_match(pred, truth):
    pred = remove_punctuation(pred.strip().lower())
    truth = remove_punctuation(truth.strip().lower())
    return int(pred == truth)

In [50]:
# Function to compute F1 Score
def compute_f1(pred, truth):
    # Remove punctuation from both pred and truth
    pred = remove_punctuation(pred.strip().lower())
    truth = remove_punctuation(truth.strip().lower())

    # Split the text into words for F1 calculation
    pred_tokens = pred.split()
    truth_tokens = truth.split()

    # Calculate the number of common tokens between the prediction and the truth
    common_tokens = set(pred_tokens) & set(truth_tokens)

    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return 0.0

    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(truth_tokens)

    if precision + recall == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)

In [51]:
# Function to compute Jaccard score
def compute_jaccard(pred, truth):
    # Normalize and remove punctuation
    pred = remove_punctuation(pred.strip().lower())
    truth = remove_punctuation(truth.strip().lower())

    # Convert to token sets
    pred_tokens = set(pred.split())
    truth_tokens = set(truth.split())

    # If both are empty
    if not pred_tokens and not truth_tokens:
        return 1.0

    # If either is empty
    if not pred_tokens or not truth_tokens:
        return 0.0

    intersection = pred_tokens.intersection(truth_tokens)
    union = pred_tokens.union(truth_tokens)

    return len(intersection) / len(union)

In [52]:
def tokenize_xlm_r(text):
    """Tokenize Sinhala text using XLM-R tokenizer and join tokens."""
    tokens = tokenizer.tokenize(text)
    return " ".join(tokens)  # Join tokens with space for ROUGE

In [53]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rouge2", "rouge3", "rouge4", "rougeL"], use_stemmer=False)

def compute_rouge(pred, truth):
    """Compute ROUGE-2 and ROUGE-L scores for a given prediction and truth."""
    pred_tokens = tokenize_xlm_r(pred)
    truth_tokens = tokenize_xlm_r(truth)
    scores = scorer.score(truth_tokens, pred_tokens)
    return scores['rouge2'].fmeasure, scores['rouge3'].fmeasure, scores['rouge4'].fmeasure, scores['rougeL'].fmeasure

In [54]:
# Splits Sinhala text into graphemes using the `grapheme` library.
def tokenize_graphemes(text):
    return grapheme.graphemes(text)  # Returns list of graphemes

In [55]:
def compute_bleu(pred, truth, n_gram):
    # Tokenize the prediction and truth
    pred_tokens = " ".join(tokenize_graphemes(pred))
    truth_tokens = [" ".join(tokenize_graphemes(truth))]  # Must be a list

    # Create BLEU scorer with proper settings
    bleu_scorer = BLEU(
        max_ngram_order=n_gram,
        smooth_method="exp",
        effective_order=True  # This addresses the warnings
    )
    bleu_score = bleu_scorer.sentence_score(pred_tokens, truth_tokens)
    return bleu_score.score

In [56]:
# add metric per model
def update_metrics_csv(model_name, metrics_dict, csv_file):
    # Create new DataFrame with current metrics
    new_data = {
        'model': [model_name],
        'timestamp': [datetime.now().isoformat()],
        **metrics_dict
    }
    new_df = pd.DataFrame(new_data)

    # If file exists, load and append new data
    if Path(csv_file).exists():
        existing_df = pd.read_csv(csv_file)
        updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    else:
        updated_df = new_df

    # Save to CSV
    updated_df.to_csv(csv_file, index=False)
    print(f"Metrics saved to {csv_file}")

## Validation Set Evaluation

In [57]:
# Calculate Exact Match for each row in the DataFrame
prediction['exact_match'] = prediction.apply(lambda row: compute_exact_match(row['pred_answer'], row['answer_text']), axis=1)
exact_match_score = prediction['exact_match'].mean() # percentage

# Calculate F1 Score for each row in the DataFrame
prediction['f1_scores'] = prediction.apply(lambda row: compute_f1(row['pred_answer'], row['answer_text']), axis=1)
f1_score_value = sum(prediction['f1_scores']) / len(prediction['f1_scores'])

# Apply row-wise Jaccard score
test['jaccard_scores'] = test.apply(lambda row: compute_jaccard(row['PredictionString'], row['answer_text']), axis=1)
jaccard_score_value = test['jaccard_scores'].mean()


# Print the results test set
print(f"Exact Match Score for val set: {exact_match_score:.4f}")
print(f"F1 Score for val set: {f1_score_value:.4f}")
print(f'Jaccard scores: {jaccard_score_value:.4f}')

Exact Match Score for val set: 0.5880
F1 Score for val set: 0.7239
Jaccard scores: 0.7049


In [58]:
# Calculate ROUGE-L and ROUGE-2
prediction[["rouge_2","rouge_3", "rouge_4", "rouge_L"]] = prediction.apply(
    lambda row: compute_rouge(row['pred_answer'], row['answer_text']), axis=1, result_type="expand"
)
avg_rouge_2 = prediction['rouge_2'].mean()
avg_rouge_3 = prediction['rouge_3'].mean()
avg_rouge_4 = prediction['rouge_4'].mean()
avg_rouge_L = prediction['rouge_L'].mean()

print(f"Average ROUGE-2 for Val set: {avg_rouge_2:.4f}")
print(f"Average ROUGE-3 for Val set: {avg_rouge_3:.4f}")
print(f"Average ROUGE-4 for Val set: {avg_rouge_4:.4f}")
print(f"Average ROUGE-L for Val set: {avg_rouge_L:.4f}")

Average ROUGE-2 for Val set: 0.0870
Average ROUGE-3 for Val set: 0.0167
Average ROUGE-4 for Val set: 0.0008
Average ROUGE-L for Val set: 0.2065


In [59]:
prediction['bleu_1'] = prediction.apply(lambda row: compute_bleu(row['pred_answer'], row['answer_text'], n_gram=1), axis=1)
prediction['bleu_2'] = prediction.apply(lambda row: compute_bleu(row['pred_answer'], row['answer_text'], n_gram=2), axis=1)

avg_bleu_1 = prediction['bleu_1'].mean()
avg_bleu_2 = prediction['bleu_2'].mean()

print(f"Average BLEU-1: {avg_bleu_1:.4f}")
print(f"Average BLEU-2: {avg_bleu_2:.4f}")

Average BLEU-1: 75.5436
Average BLEU-2: 74.5221


In [60]:
metrics = {
    'avg_bleu_1': avg_bleu_1,
    'avg_bleu_2': avg_bleu_2,
    'avg_rouge_2': avg_rouge_2,
    'avg_rouge_3': avg_rouge_3,
    'avg_rouge_4': avg_rouge_4,
    'avg_rouge_L': avg_rouge_L,
    'exact_match_score': exact_match_score,
    'f1_score_value': f1_score_value
}
# update_metrics_csv("XLM-R-75-metrics", metrics, csv_file = "/content/drive/MyDrive/Research_folder/outputs/Results/Main_Metric_Results_val.csv") #update model

## Test Set Evaluation


In [61]:
# Calculate Exact Match for each row in the DataFrame
test['exact_match'] = test.apply(lambda row: compute_exact_match(row['PredictionString'], row['answer_text']), axis=1)
exact_match_score = test['exact_match'].mean() # percentage

# Calculate F1 Score for each row in the DataFrame
test['f1_scores'] = test.apply(lambda row: compute_f1(row['PredictionString'], row['answer_text']), axis=1)
f1_score_value = sum(test['f1_scores']) / len(test['f1_scores'])

# Apply row-wise Jaccard score
test['jaccard_scores'] = test.apply(lambda row: compute_jaccard(row['PredictionString'], row['answer_text']), axis=1)
jaccard_score_value = test['jaccard_scores'].mean()


# Print the results test set
print(f"Exact Match Score for test set: {exact_match_score:.4f}")
print(f"F1 Score for test set: {f1_score_value:.4f}")
print(f'Jaccard scores: {jaccard_score_value:.4f}')

Exact Match Score for test set: 0.6048
F1 Score for test set: 0.7359
Jaccard scores: 0.7049


In [62]:
# Calculate ROUGE-L and ROUGE-2
test[["rouge_2","rouge_3", "rouge_4", "rouge_L"]] = test.apply(
    lambda row: compute_rouge(row['PredictionString'], row['answer_text']), axis=1, result_type="expand"
)
avg_rouge_2 = test['rouge_2'].mean()
avg_rouge_3 = test['rouge_3'].mean()
avg_rouge_4 = test['rouge_4'].mean()
avg_rouge_L = test['rouge_L'].mean()

print(f"Average ROUGE-2 for Val set: {avg_rouge_2:.4f}")
print(f"Average ROUGE-3 for Val set: {avg_rouge_3:.4f}")
print(f"Average ROUGE-4 for Val set: {avg_rouge_4:.4f}")
print(f"Average ROUGE-L for Val set: {avg_rouge_L:.4f}")

Average ROUGE-2 for Val set: 0.0857
Average ROUGE-3 for Val set: 0.0216
Average ROUGE-4 for Val set: 0.0032
Average ROUGE-L for Val set: 0.2114


In [63]:
# Calculate BLEU-1 and BLEU-2
test['bleu_1'] = test.apply(lambda row: compute_bleu(row['PredictionString'], row['answer_text'], n_gram=1), axis=1)
test['bleu_2'] = test.apply(lambda row: compute_bleu(row['PredictionString'], row['answer_text'], n_gram=2), axis=1)

avg_bleu_1 = test['bleu_1'].mean()
avg_bleu_2 = test['bleu_2'].mean()

# Print the results test set
print(f"Average BLEU-1 for Test set: {avg_bleu_1:.4f}")
print(f"Average BLEU-2 for Test set: {avg_bleu_2:.4f}")

Average BLEU-1 for Test set: 76.6579
Average BLEU-2 for Test set: 75.6876


In [64]:
metrics = {
    'avg_bleu_1': avg_bleu_1,
    'avg_bleu_2': avg_bleu_2,
    'avg_rouge_2': avg_rouge_2,
    'avg_rouge_3': avg_rouge_3,
    'avg_rouge_4': avg_rouge_4,
    'avg_rouge_L': avg_rouge_L,
    'exact_match_score': exact_match_score,
    'f1_score_value': f1_score_value,
    'jaccard_score_value': jaccard_score_value
}
# update_metrics_csv("XLM-R-75", metrics, csv_file = "/content/drive/MyDrive/Research_folder/outputs/Results/Evaluation_Results_Test.csv") #update model

## Inference Process

In [70]:
# Load the fine-tuned model and tokenizer
model_name = "/content/drive/MyDrive/Research_folder/models/Sin-xlm-r-Large_multi5"  # Update with your actual model path
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [71]:
# Example of using the model for inference
question = "රිට් පෙත්සම ඉදිරිපත් කරේ කවුද?"
context = '2023 වසරේ මාතර, වැලිගම ප්‍රදේශයේ සිදුවු වෙඩිතැබීමට අදාළව තමා අත්අඩංගුවට ගැනීමට මාතර මහෙස්ත්‍රාත් අධිකරණය ලබාදුන් නියෝගය ක්‍රියාත්මක කිරීම වළක්වාලමින් අතුරුතහනම් නියෝගයක් නිකුත් කරන්නැයි ඉල්ලා දේශබන්දු තෙන්නකෝන් මහතා ඉදිරිපත් කළ රිට් පෙත්සම විභාගයට නොගෙනම නිෂ්ප්‍රභ කිරීමට අභියාචනාධිකරණය අද (17) තීන්දු කළේය. ඒ අනුව දේශබන්දු තෙන්නකෝන් මහතා වහාම අත්අඩංගුවට ගෙන අධිකරණයට ඉදිරිපත් කිරීමට පියවර ගන්නා ලෙසත් විනිසුරු මඩුල්ල විසින් අපරාධ පරීක්ෂණ දෙපාර්තමේන්තුවට නියෝග කරනු ලැබීය. දේශබන්දු තෙන්නකෝන් මහතා විසින් ඉදිරිපත් කළ රිට් පෙත්සමක් ඉකුත් 12 වනදා අභියාචනාධීකරණය හමුවේ සලකා බැලුණි. එහි තීන්දුව අද පෙරවරුවේ ප්‍රකාශයට පත් කළ අභියාචනාධිකරණයේ වැඩබලන සභාපති විනිසුරු මොහොමඩ් ලෆාර් තාහීර් සහ සරත් දිසානායක යන මහත්වරුන්ගෙන් සමන්විත අභියාචනාධිකරණ විනිසුරු මඩුල්ල විසින් මෙම නියෝගය නිකුත් කළහ.'

inputs = tokenizer(question, context, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

start_idx = outputs.start_logits.argmax()
end_idx = outputs.end_logits.argmax()

answer_tokens = inputs.input_ids[0][start_idx:end_idx+1]
answer = tokenizer.decode(answer_tokens)
print(f"Question: {question}")
print(f"Answer: {answer}")


Question: රිට් පෙත්සම ඉදිරිපත් කරේ කවුද?
Answer: දේශබන්දු තෙන්නකෝන්


## Simple UI design

In [72]:
def answer_question(context, question):
    # Tokenize the question and context pair for model input, with truncation for long inputs
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)

    # Disable gradient computation since only need inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the start and end logits for the predicted answer span
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Get the most probable start and end indices for the answer span
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1

    # Convert the token indices to the corresponding text answer
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index]))

    return answer

In [79]:
# simple Gradio interface for demo
iface = gr.Interface(
    fn=answer_question,
    inputs=[gr.Textbox(label="Context"), gr.Textbox(label="Question")],
    outputs="text",
    title="Sinhala Question Answering System",
    description="Enter a passage and a question to get an answer."
)
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8499463c44a259dad1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
